<a href="https://colab.research.google.com/github/flp-cmd/Notebooks_ML/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [342]:
data = pd.read_csv("train.csv")

In [343]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [344]:
data.corr()['Survived'].sort_values(ascending=False)

<ipython-input-344-025bedf64b2d>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()['Survived'].sort_values(ascending=False)


Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Name: Survived, dtype: float64

In [345]:
from sklearn.model_selection import StratifiedShuffleSplit

In [364]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(data, data[['Survived', 'Pclass', 'Sex']]):
  strat_train_set = data.loc[train_index]
  strat_test_set = data.loc[test_index]

In [347]:
strat_train_set['Survived'].value_counts()/len(strat_train_set)

0    0.616372
1    0.383628
Name: Survived, dtype: float64

In [348]:
strat_test_set['Survived'].value_counts()/len(strat_test_set)

0    0.615672
1    0.384328
Name: Survived, dtype: float64

In [349]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

In [251]:
class AgeImputer(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    imputer = SimpleImputer(strategy = 'mean')
    X['Age'] = imputer.fit_transform(X[['Age']])
    return X

In [252]:
from sklearn.preprocessing import OneHotEncoder

class CategoryEncoder(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    X['Embarked'].fillna('S',inplace=True)
    ohe = OneHotEncoder()
    matrix_cat = ohe.fit_transform(X[['Sex','Embarked']]).toarray()
    features = pd.DataFrame(data = matrix_cat, columns=['female', 'male', 'C', 'Q', 'S'])
    X = X.reset_index().drop('index',axis=1)
    X = pd.concat([X, features],axis=1)
    return X

In [253]:
class FeatureDropper(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    return X.drop(['Embarked','Sex','Name','Ticket','Cabin'], axis=1)

In [370]:
class FeatureScalingSplitting(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    scaler = StandardScaler()
    X_transformed = X.drop('Survived',axis=1)
    y_transformed = X['Survived']

    X_transformed = scaler.fit_transform(X_transformed)
    y_transformed = y_transformed.to_numpy()

    return X_transformed, y_transformed

In [365]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('ageImputer', AgeImputer()),
                     ("categoryEncoder", CategoryEncoder()),
                     ("featureDropper", FeatureDropper())])

In [367]:
strat_train_set.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,695,0,1,60.000000,0,0,26.5500,0.0,1.0,0.0,0.0,1.0
1,884,0,2,28.000000,0,0,10.5000,0.0,1.0,0.0,0.0,1.0
2,525,0,3,29.765903,0,0,7.2292,0.0,1.0,1.0,0.0,0.0
3,330,1,1,16.000000,0,1,57.9792,1.0,0.0,1.0,0.0,0.0
4,829,1,3,29.765903,0,0,7.7500,0.0,1.0,0.0,1.0,0.0


In [366]:
strat_train_set = pipeline.fit_transform(strat_train_set)

In [257]:
strat_train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Age          712 non-null    float64
 4   SibSp        712 non-null    int64  
 5   Parch        712 non-null    int64  
 6   Fare         712 non-null    float64
 7   female       712 non-null    float64
 8   male         712 non-null    float64
 9   C            712 non-null    float64
 10  Q            712 non-null    float64
 11  S            712 non-null    float64
dtypes: float64(7), int64(5)
memory usage: 66.9 KB


In [377]:
from sklearn.model_selection import cross_val_score

def test_size_opt(model):
    scores_max = 0.01
    test_size_var = 0.05
    while test_size_var <= 0.6:
        split = StratifiedShuffleSplit(n_splits=1, test_size=test_size_var, random_state=42)
        for train_index, test_index in split.split(data, data[['Survived', 'Pclass', 'Sex']]):
          strat_train_set = data.loc[train_index]
          strat_test_set = data.loc[test_index]
        strat_train_set = pipeline.fit_transform(strat_train_set)
        fss = FeatureScalingSplitting()
        X_train, y_train = fss.fit_transform(strat_train_set)
        scores = cross_val_score(model, X_train, y_train, cv=10)
        test_size_var += 0.05
        if scores.mean() > scores_max:
            scores_max = scores.mean()
            best_test_size = test_size_var
    print(f"Melhor test_size: {best_test_size:.2f}\nMáxima média de score: {scores_max:.2f}")

In [378]:
test_size_opt(rfc)

Melhor test_size: 0.25
Máxima média de score: 0.83


In [380]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data, data[['Survived', 'Pclass', 'Sex']]):
  strat_train_set = data.loc[train_index]
  strat_test_set = data.loc[test_index]

In [381]:
strat_train_set = pipeline.fit_transform(strat_train_set)
fss = FeatureScalingSplitting()
X_train, y_train = fss.fit_transform(strat_train_set)

In [263]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [384]:
rfc = RandomForestClassifier()
grid_params = {"n_estimators":[300,350,400], "max_depth":[8,10,12], "min_samples_split":[10,11,12]}

grid_search_rfc = GridSearchCV(rfc, grid_params, cv=5, scoring='accuracy')
grid_search_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [8, 10, 12],
                         'min_samples_split': [10, 11, 12],
                         'n_estimators': [300, 350, 400]},
             scoring='accuracy')

In [385]:
best_rfc_model = grid_search_rfc.best_estimator_

In [386]:
best_rfc_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=12, min_samples_split=10, n_estimators=350)

In [387]:
best_rfc_model.score(X_train, y_train)

0.9176646706586826

In [388]:
strat_test_set = pipeline.fit_transform(strat_test_set)
X_test, y_test = fss.fit_transform(strat_test_set)

In [389]:
best_rfc_model.score(X_test, y_test)

0.8116591928251121

In [390]:
y_pred = best_rfc_model.predict(X_test)

In [391]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       137
           1       0.80      0.69      0.74        86

    accuracy                           0.81       223
   macro avg       0.81      0.79      0.80       223
weighted avg       0.81      0.81      0.81       223



In [392]:
data_formatted = pipeline.fit_transform(data)
X_final, y_final = fss.fit_transform(data_formatted)

In [408]:
final_rfc = RandomForestClassifier()
grid_params = {"n_estimators":[600,630,650], "max_depth":[15,16,17]}

grid_search_rfc_final = GridSearchCV(final_rfc, grid_params, cv=5, scoring='accuracy')

grid_search_rfc_final.fit(X_final, y_final)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [15, 16, 17],
                         'n_estimators': [600, 630, 650]},
             scoring='accuracy')

In [409]:
final_best_rfc = grid_search_rfc_final.best_estimator_

In [410]:
final_best_rfc.fit(X_final, y_final)

RandomForestClassifier(max_depth=15, n_estimators=630)

In [411]:
test_data = pd.read_csv("test.csv")

In [412]:
X_final_test = pipeline.fit_transform(test_data)

In [413]:
X_final_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age          418 non-null    float64
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   Fare         417 non-null    float64
 6   female       418 non-null    float64
 7   male         418 non-null    float64
 8   C            418 non-null    float64
 9   Q            418 non-null    float64
 10  S            418 non-null    float64
dtypes: float64(7), int64(4)
memory usage: 36.0 KB


In [414]:
X_final_test = X_final_test.fillna(method='ffill')

scaler = StandardScaler()

X_final_test = scaler.fit_transform(X_final_test)

In [415]:
final_prediction = final_best_rfc.predict(X_final_test)

In [416]:
final_df = pd.DataFrame(test_data['PassengerId'])
final_df['Survived'] = final_prediction

In [417]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [419]:
final_df.to_csv("final_predictions2.csv", index=False)